## Segmenting and Clustering Neighborhoods in Toronto - Explore and Cluster

In [1]:
import pandas as pd

In [2]:
latlong = pd.read_csv("http://cocl.us/Geospatial_data/") 
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [3]:
#Rename "Postal Code" with "PostalCode"
latlong.columns = ['PostalCode','Latitude','Longitude']
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Make sure there are 103 postal codes as we determined from previous assignment

In [5]:
latlong.shape[0]

103

#### We use the code from the Notebook Toronto Wiki Scraping.ipynb to get the postcode, borough and neigborhood (previous assignment

In [6]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(url, header=0,keep_default_na=False) 
table_df = table[0]
table_df.columns = ['PostalCode','Borough','Neighborhood']
table_df = table_df.query('Borough != "Not assigned"').reset_index(drop=True)
table_df1=table_df.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
table_df1.loc[table_df1['Neighborhood'] == 'Not assigned', 'Neighborhood' ] = table_df1['Borough']
table_df1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Now, we merge the two tables with latlong on the right

In [8]:
table_df2 = pd.merge(left=table_df1, right=latlong, on="PostalCode", how="right")
table_df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Now, we merge the two tables with latlong on the right

In [9]:
table_df2 = pd.merge(left=table_df1, right=latlong, on="PostalCode", how="right")
table_df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Let us now explore Toronto

#### Map of Toronto with neighborhoods superimposed

In [16]:
!pip install folium

     |████████████████████████████████| 92kB 13.9MB/s eta 0:00:01


In [19]:
import folium
# create map of York using latitude and longitude values

map_Toronto = folium.Map(location=[43.761539, -79.411079], zoom_start=11)

# add markers to map
for lat, lng, bor, nei in zip(table_df2['Latitude'], table_df2['Longitude'], table_df2['Borough'], table_df2['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Toronto)  
    
map_Toronto

#### Define Foursquare Credentials and Version

In [20]:
###Removing CLIENT_ID and CLIENT_SECRET after run

CLIENT_ID = '....' # your Foursquare ID
CLIENT_SECRET = '.....' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Examine Neighborhoods in Scarborough, lat-long 43.773077, -79.257774

In [21]:
Scarborough_data = table_df2[table_df2['Borough'] == 'Scarborough'].reset_index(drop=True)
map_scarborough = folium.Map(location=[43.773077, -79.257774], zoom_start=11)
for lat, lng, bor, nei in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'],Scarborough_data['Borough'], Scarborough_data['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarborough)  
    
map_scarborough

#### We find nearby venues in Scarborough

In [22]:
import requests
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)


scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )


Malvern, Rouge
Highland Creek, Port Union, Rouge Hill
Morningside, Guildwood, West Hill
Woburn
Cedarbrae
Scarborough Village
Ionview, East Birchmount Park, Kennedy Park
Golden Mile, Oakridge, Clairlea
Cliffside, Scarborough Village West, Cliffcrest
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Agincourt North, Steeles East, Milliken, L'Amoreaux East
L'Amoreaux West
Upper Rouge


#### Now we cluster and visulaize neighborhoods

In [23]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Steeles East, Milliken, L'Amoreaux East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clarks Corners, Tam O'Shanter, Sullivan",9,9,9,9,9,9
"Cliffside, Scarborough Village West, Cliffcrest",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",7,7,7,7,7,7
"Golden Mile, Oakridge, Clairlea",10,10,10,10,10,10
"Highland Creek, Port Union, Rouge Hill",1,1,1,1,1,1


In [24]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,...,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.0,0.000000,0.00,...,0.000000,0.000,0.000000,0.25,0.00,0.0,0.000,0.000000,0.000000,0.000000
1,"Agincourt North, Steeles East, Milliken, L'Amo...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,...,0.333333,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.25,...,0.000000,0.000,0.000000,0.25,0.00,0.0,0.000,0.000000,0.000000,0.000000
3,Cedarbrae,0.000000,0.142857,0.142857,0.142857,0.0,0.000000,0.0,0.000000,0.00,...,0.000000,0.000,0.000000,0.00,0.00,0.0,0.000,0.142857,0.000000,0.000000
4,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000000,0.000000,0.111111,0.0,0.000000,0.0,0.000000,0.00,...,0.000000,0.000,0.000000,0.00,0.00,0.0,0.000,0.111111,0.000000,0.000000
5,"Cliffside, Scarborough Village West, Cliffcrest",0.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,...,0.000000,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.000000,0.000000
6,"Dorset Park, Scarborough Town Centre, Wexford ...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,...,0.000000,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.000000,0.142857
7,"Golden Mile, Oakridge, Clairlea",0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.100000,0.00,...,0.000000,0.000,0.000000,0.00,0.00,0.1,0.000,0.000000,0.000000,0.000000
8,"Highland Creek, Port Union, Rouge Hill",0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.000000,0.00,...,0.000000,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.000000,0.000000
9,"Ionview, East Birchmount Park, Kennedy Park",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.166667,0.00,...,0.000000,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.000000,0.000000


In [25]:
import numpy as np
num_top_venues = 17

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Vietnamese Restaurant,Hobby Shop,Chinese Restaurant
1,"Agincourt North, Steeles East, Milliken, L'Amo...",Park,Playground,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Vietnamese Restaurant,Hobby Shop,Chinese Restaurant,Caribbean Restaurant
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Clothing Store,Indian Restaurant,Chinese Restaurant
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Convenience Store,Grocery Store,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Clothing Store,College Stadium
4,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Chinese Restaurant,Thai Restaurant,Italian Restaurant,Bank,Fried Chicken Joint,Fast Food Restaurant,Noodle House,Coffee Shop,General Entertainment,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Vietnamese Restaurant,Clothing Store
5,"Cliffside, Scarborough Village West, Cliffcrest",American Restaurant,Motel,Indian Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant
6,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Light Rail Station,Pet Store,Vietnamese Restaurant,Breakfast Spot,Department Store,Grocery Store,General Entertainment,Fried Chicken Joint,Athletics & Sports,Bakery,Fast Food Restaurant,Electronics Store,Discount Store,Convenience Store
7,"Golden Mile, Oakridge, Clairlea",Bus Line,Bakery,Intersection,Fast Food Restaurant,Metro Station,Bus Station,Park,Soccer Field,Breakfast Spot,Bar,Grocery Store,General Entertainment,Fried Chicken Joint,Athletics & Sports,Electronics Store,Discount Store,Department Store
8,"Highland Creek, Port Union, Rouge Hill",Bar,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Clothing Store,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant
9,"Ionview, East Birchmount Park, Kennedy Park",Discount Store,Hobby Shop,Coffee Shop,Bus Station,Department Store,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Convenience Store,College Stadium,Vietnamese Restaurant,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant


#### Cluster the neighborhood Scarborough

In [26]:
from sklearn.cluster import KMeans

# Number of clusters 4
# Using k-means to cluster the neighborhood into 4 clusters.
kclusters = 4

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 3, 1, 1], dtype=int32)

In [27]:
scarborough_merged = Scarborough_data[0:16]

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,...,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Clothing Store,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant,Café
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497,1,Bar,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,...,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Clothing Store,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711,1,Intersection,Spa,Electronics Store,Breakfast Spot,...,Mexican Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,College Stadium,Chinese Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Indian Restaurant,Korean Restaurant,Hakka Restaurant,...,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Vietnamese Restaurant,Clothing Store,Chinese Restaurant,Caribbean Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,...,Vietnamese Restaurant,Convenience Store,Grocery Store,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Clothing Store,College Stadium
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,...,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium,Clothing Store,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant
6,M1K,Scarborough,"Ionview, East Birchmount Park, Kennedy Park",43.727929,-79.262029,1,Discount Store,Hobby Shop,Coffee Shop,Bus Station,...,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Convenience Store,College Stadium,Vietnamese Restaurant,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea",43.711112,-79.284577,1,Bus Line,Bakery,Intersection,Fast Food Restaurant,...,Soccer Field,Breakfast Spot,Bar,Grocery Store,General Entertainment,Fried Chicken Joint,Athletics & Sports,Electronics Store,Discount Store,Department Store
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest",43.716316,-79.239476,0,American Restaurant,Motel,Indian Restaurant,Hakka Restaurant,...,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1,General Entertainment,Skating Rink,Café,College Stadium,...,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Clothing Store,Indian Restaurant,Chinese Restaurant


In [28]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[43.773077, -79.257774], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


### Examine Clusters

#### Cluster 0

In [30]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
8,"Cliffside, Scarborough Village West, Cliffcrest",0,American Restaurant,Motel,Indian Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant


#### Cluster 1

In [31]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,"Malvern, Rouge",1,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Clothing Store,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant,Café
1,"Highland Creek, Port Union, Rouge Hill",1,Bar,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Clothing Store,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant
2,"Morningside, Guildwood, West Hill",1,Intersection,Spa,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,College Stadium,Chinese Restaurant
3,Woburn,1,Coffee Shop,Indian Restaurant,Korean Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Vietnamese Restaurant,Clothing Store,Chinese Restaurant,Caribbean Restaurant
4,Cedarbrae,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Convenience Store,Grocery Store,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Clothing Store,College Stadium
5,Scarborough Village,1,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium,Clothing Store,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant
6,"Ionview, East Birchmount Park, Kennedy Park",1,Discount Store,Hobby Shop,Coffee Shop,Bus Station,Department Store,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Convenience Store,College Stadium,Vietnamese Restaurant,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant
7,"Golden Mile, Oakridge, Clairlea",1,Bus Line,Bakery,Intersection,Fast Food Restaurant,Metro Station,Bus Station,Park,Soccer Field,Breakfast Spot,Bar,Grocery Store,General Entertainment,Fried Chicken Joint,Athletics & Sports,Electronics Store,Discount Store,Department Store
9,"Birch Cliff, Cliffside West",1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Clothing Store,Indian Restaurant,Chinese Restaurant
10,"Dorset Park, Scarborough Town Centre, Wexford ...",1,Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Light Rail Station,Pet Store,Vietnamese Restaurant,Breakfast Spot,Department Store,Grocery Store,General Entertainment,Fried Chicken Joint,Athletics & Sports,Bakery,Fast Food Restaurant,Electronics Store,Discount Store,Convenience Store


#### Cluster 2

In [32]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
11,"Wexford, Maryvale",2,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Clothing Store,Coffee Shop,Hobby Shop


#### Cluster 3

In [33]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
13,"Clarks Corners, Tam O'Shanter, Sullivan",3,Pizza Place,Chinese Restaurant,Thai Restaurant,Italian Restaurant,Bank,Fried Chicken Joint,Fast Food Restaurant,Noodle House,Coffee Shop,General Entertainment,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Vietnamese Restaurant,Clothing Store


#### Thats all !!!